In [122]:
""" Playgame routine.  Plays one game between StratA and StratB.  Outputs return to A
    Added output of cards and plays to allow ML"""

def playgame(GameDeck, StratA, StratB, verbose = False):

    # Deal 
    cardA = GameDeck.deal()
    cardB = GameDeck.deal()
    if verbose: print("Card A: ", cardA, " Card B: ", cardB)
    playA = ""
    playB = ""
    
    # Player A decides
    playA = StratA.play(cardA,"A")
    if verbose: print("Player A: ", playA)

    # if Player A pass, showdown for $2    
    if playA == "Pass":
        if cardA > cardB:
            payout = 1
        elif cardB > cardA:
            payout = -1
        else:
            payout = 0
    # if Player A raises, player B decides
    else:
        playB = StratB.play(cardB,"B")
        if verbose: print("Player B: ", playB)
        
        #if player B calls, showdown for $4
        if playB == "Call":
            if cardA > cardB:
                payout = 2
            elif cardB > cardA:
                payout = -2
            else:
                assert (cardA == cardB)
                payout = 0
        # if player B folds, A gets the ante
        else:
            payout = 1
    if verbose: 
        print("Payout: ",payout)
        print("")
    return {'winA':payout,'playA':playA,'playB':playB,'cardA':cardA,'cardB':cardB}
        

In [3]:
""" Deck class  Defines the deck.  For now only a discrete set 0 - n-1"""

class Deck:
    def __init__(self, decksize):
        self.decksize = decksize
        self.cards = range(self.decksize)      
        
    def deal(self):
        import random              # is it ok to have this here?
        card_delt = random.randint(0,self.decksize -1 )
        return card_delt
    

In [8]:
"""  Strategy Class.  Sets standards for all strategies 
    Create a subclass for each strategy"""

class Strategy:
    def __init__(self):
        pass
        # self.gamedeck = GameDeck
        # self.decksize = GameDeck.decksize
        
    def play(self,mycard,player):
        """ determine strategy for player, having been dealt card mycard.  
        If player = "A" return either 'Pass' or 'Raise' 
        If player = 'B' return either 'Fold' or 'Call' """ 
        pass
    



In [21]:
""" Vector based strategy.  

    Paramaterized by a vector giving probability of aggressive (Raise/Call) strategy for each card.
    
    Old strategies
    
    Random - [1/2,1/2,1/2,1/2,1/2,1/2]
    Simple - e.g [0,0,0,1,1,1] 
    bluff - e.g [p,p,p,1,1,1]
    optimal - A [2/3,0,0,0,1,1]  B [0, 1/3,1/3,1,1,1]"""

class vectorstrat(Strategy):
    
    def __init__(self,aggprobs):
        self.aggprobs = aggprobs
        
    def play(self,mycard,player):
        import random
        if random.random() > self.aggprobs[mycard]:
            if player == "A":
                return 'Pass'
            else:
                return 'Fold'
        else:
            if player == "A":
                return "Raise"
            else:
                return 'Call'

        

In [39]:
"""Challenge Routine.
Plays n games between two strategies and returns the net result.
Used for testing
"""

def challenge(num_games,strata,stratb,strataname = "",stratbname = "",verbose = False):

    decksize = 6
    d = Deck(decksize)

    a_net_wins = 0

    if verbose: print("Player A: ", strataname,"   Player B: ", stratbname )

    for i in range (num_games):
        a_net_wins += playgame(d,strata,stratb)['winA']
        if verbose and i % 1000 == 0: print(i," games played")

    if verbose: print(strataname, " won $", a_net_wins, "  $", a_net_wins / num_games, " per game.")
    
    return a_net_wins

In [367]:
""" First cut learning.
    Will use logistic regression to set each of the vector parameters individually.
    
    Inline training -- play one game, instantly update.
    
    Train only A player.
    
    """
import math
def logit(x):
    p = 1 / ( 1 + math.exp(-x))
    return p

def logodds(p):
    if p == 0: return -10
    elif p == 1: return 10
    else:
        x = math.log(p / (1-p))
        return x


def train_A_logit(num_games = 10 ** 6,num_updates = 20 , alpha = 0.005, decksize = 6,
                  start_strategy =[], strategyB =[],verbose = False,learnvsaverage = False,diag = False,pass_win_reward=-1):

    import random
    import copy
    
    # set paramaters and counters

    d = Deck(decksize)
    
    if start_strategy == [] :
        paramvector = [0 for i in range(decksize)]  # starts random parameter = 0 -> prob 50%
    else:
        paramvector = [logodds(p) for p in start_strategy]
        
    

    if strategyB == []:   # default strategy B is fold for lower half, call for larger half
        strategyB = [i > decksize / 2.0 for i in range(decksize)]
    sb = vectorstrat(strategyB)
    winnings = 0
    winnings_temp = 0
    training_updates =[]
    games_update = num_games / num_updates
    
    
    for i in range(num_games+1):
        
        # set strategy to current logit of parameters
        strategyvector = [logit(x) for x in paramvector]
        sa = vectorstrat(strategyvector)

        #play a game
        result = playgame(d,sa,sb,verbose = False)
        if diag and (result['cardA'] == 5): print("Game: ",i,"Result:",result)
        
        # direction of update is positive for a plus action.  Maybe change action from string to +/- 1
        if result['playA'] == "Raise":
            direction = 1
        else:
            if result['winA'] > 0 :
                direction =  - pass_win_reward
            else:
                direction = -1
        
        # update strategy.  Alpha is change rate parameter.  
        # Result * direction positive when aggression paid off or passive resulted in loss -> increase prob.
        # idea -- change result to result - average
        avgwins = 0
        if learnvsaverage:
            avgwins = winnings / i
            
        paramvector[result['cardA']] += alpha * (result['winA']-avgwins) *  direction
        
      
        # avoid over/underflow errors  Likely a better method, but this works within 1%
        paramvector[result['cardA']] = max(-10,paramvector[result['cardA']])
        paramvector[result['cardA']] = min(10,paramvector[result['cardA']])
        
        if (diag and result['cardA'] == 5): print(paramvector[5]) 
        
        # track performance
        winnings += result['winA']
        winnings_temp += result['winA']
 
        # periodically update
        if i % games_update == 0 :
            if verbose:
                if i > 0: 
                    tempwinrate = round(winnings_temp / games_update,4) 
                else: 
                    tempwinrate = "       "
                print(i, tempwinrate,[round(logit(x)*100,2) for x in paramvector])    
            training_updates += [{'games':i,'tempwins':winnings_temp,'logodds':copy.deepcopy(paramvector)}]
            winnings_temp = 0
    
    if verbose:
        print("Done")
        winrate = winnings / num_games
        print("Winrate = ", winrate)
        print([round(x,4) for x in paramvector])
        strategyvector = [round(logit(x)*100,2)  for x in paramvector]
        print(strategyvector)
        
    return {'num_games':num_games, 'winnings':winnings,'final model':paramvector,'training_updates':training_updates}
        
    



Added some diagnostic printing, now running for a small test. First staring in center.

In [360]:
t = train_A_logit(num_games = 100000,num_updates = 20 , alpha = 0.005, decksize = 6,
                  start_strategy =[], strategyB =[],verbose = False,learnvsaverage = False,diag=True)

Game:  1 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-0.005
Game:  2 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
0.0
Game:  3 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-0.005
Game:  4 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
0.005
Game:  8 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
0.01
Game:  14 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
0.005
Game:  18 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
0.0
Game:  23 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-0.005
Game:  24 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-0.01
Game:  27 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
-0.005
Game:  32 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 

Game:  637 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
0.015000000000000013
Game:  639 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
0.010000000000000012
Game:  647 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
0.02000000000000001
Game:  650 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
0.025000000000000012
Game:  652 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
0.030000000000000013
Game:  655 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
0.03500000000000001
Game:  657 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
0.03000000000000001
Game:  659 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
0.03500000000000001
Game:  667 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
0.04500000000000001
Game:  668 Result: {'

Game:  2219 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
0.22000000000000008
Game:  2241 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
0.21500000000000008
Game:  2242 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
0.22000000000000008
Game:  2252 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
0.21500000000000008
Game:  2262 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
0.22000000000000008
Game:  2272 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
0.21500000000000008
Game:  2275 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
0.21000000000000008
Game:  2276 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
0.21500000000000008
Game:  2280 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
0.21000000000000008
Game:  2286 Result: {'winA

Game:  3918 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
0.5600000000000004
Game:  3919 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
0.5700000000000004
Game:  3941 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
0.5700000000000004
Game:  3944 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
0.5750000000000004
Game:  3946 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
0.5800000000000004
Game:  3954 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
0.5750000000000004
Game:  3974 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
0.5800000000000004
Game:  3983 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
0.5900000000000004
Game:  3984 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
0.5950000000000004
Game:  3985 Res

1.254999999999998
Game:  5487 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
1.2599999999999978
Game:  5511 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
1.2699999999999978
Game:  5520 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
1.2749999999999977
Game:  5527 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
1.2699999999999978
Game:  5529 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
1.2749999999999977
Game:  5532 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
1.2799999999999976
Game:  5550 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
1.2799999999999976
Game:  5562 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
1.2799999999999976
Game:  5566 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
1.27999999999999

...

Game:  92580 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  92587 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  92588 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  92594 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  92595 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  92597 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
10
Game:  92618 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  92622 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  92624 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
10
Game:  92625 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  92628 Result: {'winA': 1, 'playA': 'Raise',

Game:  93854 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
10
Game:  93856 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
10
Game:  93861 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
10
Game:  93865 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
10
Game:  93870 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  93881 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
10
Game:  93884 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
10
Game:  93885 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  93889 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  93896 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
10
Game:  93898 Result: {'winA': 1, 'playA': 'Raise',

10
Game:  95759 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  95760 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
10
Game:  95791 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  95792 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  95793 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
10
Game:  95794 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
10
Game:  95807 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  95810 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  95820 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  95823 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  95827 Result: {'winA': 0, 'playA': 'Rais

Game:  97732 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  97734 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
10
Game:  97743 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
10
Game:  97751 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
10
Game:  97756 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
10
Game:  97760 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 1}
10
Game:  97763 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  97773 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
10
Game:  97775 Result: {'winA': 2, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 4}
10
Game:  97777 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  97786 Result: {'winA': 1, 'playA': 'Raise',

Game:  99400 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  99403 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  99405 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  99407 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 0}
10
Game:  99409 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  99417 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 3}
10
Game:  99424 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  99427 Result: {'winA': 1, 'playA': 'Raise', 'playB': 'Fold', 'cardA': 5, 'cardB': 2}
10
Game:  99432 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
10
Game:  99433 Result: {'winA': 0, 'playA': 'Raise', 'playB': 'Call', 'cardA': 5, 'cardB': 5}
10
Game:  99458 Result: {'winA': 1, 'playA': 'Raise',

Seems OK.  Now try starting in corner

In [362]:
t = train_A_logit(num_games = 100000,num_updates = 20 , alpha = 0.005, decksize = 6,
                  start_strategy =[.01,.01,.01,.01,.01,.01], strategyB =[],verbose = False,learnvsaverage = False,diag=True)

Game:  1 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-4.60011985013459
Game:  5 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-4.60511985013459
Game:  10 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-4.60511985013459
Game:  11 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-4.6101198501345895
Game:  16 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-4.615119850134589
Game:  31 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-4.615119850134589
Game:  45 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-4.620119850134589
Game:  65 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-4.620119850134589
Game:  73 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-4.625119850134589
Game:  79 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'card

Game:  659 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-5.07011985013458
Game:  665 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-5.07511985013458
Game:  670 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-5.0801198501345795
Game:  683 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-5.085119850134579
Game:  684 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-5.085119850134579
Game:  688 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-5.090119850134579
Game:  702 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-5.095119850134579
Game:  706 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-5.100119850134579
Game:  712 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-5.105119850134579
Game:  718 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'car

Game:  2295 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-6.2051198501345555
Game:  2305 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-6.210119850134555
Game:  2313 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-6.215119850134555
Game:  2315 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-6.220119850134555
Game:  2321 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-6.225119850134555
Game:  2323 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-6.230119850134555
Game:  2332 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-6.235119850134555
Game:  2334 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-6.235119850134555
Game:  2335 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-6.240119850134555
Game:  2336 Result: {'winA': 1, 'playA': 'Pass', 'play

Game:  4085 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-7.4251198501345295
Game:  4089 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-7.4251198501345295
Game:  4097 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-7.4251198501345295
Game:  4102 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-7.430119850134529
Game:  4111 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-7.435119850134529
Game:  4115 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-7.440119850134529
Game:  4116 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-7.440119850134529
Game:  4117 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-7.445119850134529
Game:  4125 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-7.450119850134529
Game:  4130 Result: {'winA': 1, 'playA': 'Pass', 'pl

Game:  5734 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-8.580119850134608
Game:  5738 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-8.585119850134609
Game:  5739 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-8.59011985013461
Game:  5751 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-8.59511985013461
Game:  5755 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-8.59511985013461
Game:  5756 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-8.600119850134611
Game:  5759 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-8.605119850134612
Game:  5768 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-8.610119850134613
Game:  5774 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-8.615119850134613
Game:  5778 Result: {'winA': 1, 'playA': 'Pass', 'playB': 

...

Game:  92242 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  92257 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-10
Game:  92258 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  92267 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-10
Game:  92270 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  92271 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  92274 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  92277 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  92285 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  92289 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10
Game:  92295 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-10

Game:  93947 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  93952 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10
Game:  93954 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  93955 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  93960 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  93965 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  93966 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  93972 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10
Game:  93973 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-10
Game:  93981 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-10
Game:  93986 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10

-10
Game:  95629 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  95630 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-10
Game:  95637 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  95638 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  95641 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  95642 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-10
Game:  95646 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-10
Game:  95652 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  95655 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-10
Game:  95662 Result: {'winA': 0, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 5}
-10
Game:  95664 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}

Game:  97269 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  97276 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-10
Game:  97288 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  97293 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  97298 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10
Game:  97303 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  97312 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  97315 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10
Game:  97316 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10
Game:  97317 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10
Game:  97327 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10

Game:  98614 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-10
Game:  98615 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  98617 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 3}
-10
Game:  98622 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-10
Game:  98626 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 2}
-10
Game:  98627 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  98629 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 4}
-10
Game:  98652 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10
Game:  98653 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  98673 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 1}
-10
Game:  98688 Result: {'winA': 1, 'playA': 'Pass', 'playB': '', 'cardA': 5, 'cardB': 0}
-10

Issue now clear.  If we pass on a high card, we are likley to still win $! and thus be rewarded.  We aren't penalized for the opportunity cost of not having raised (we could have won $2).  When starting in the middle, we try both and the double reward for raising outweigh the single reward for passing.  However, when starting at a low prob of raising, there aren't enough raises to get that reward.

Three options
a) Be sure to choose starting point carefully 
b) Eliminate reward for pass / win -- story is we have mixed feelings, so reinforcement = 0
c) Add negative reinforcement for pass / win -- story is that we realize we should have played differently

Added code to test b and c.  Bringing up code for compare outcomes.

In [363]:
"""Second compare.  Used to compare outcomes.

   Moving from ad-hoc review of performance to something better.


'num_games':num_games, 'winnings':winnings,'final model':paramvector,'training_updates':training_updates

"""
def compare_outcomes (args):
    
    # args array of - dict.  'name' 'decksize' 'alpha' 'initial' 'opponent'
    
    results =[]
    
    print("Name  WinRate      Probvector")
    
    for arg in args:
            
        results = train_A_logit(num_games = arg['num_games'], num_updates = arg['num_updates'],  
                             alpha = arg['alpha'], decksize = arg['decksize'],
                             start_strategy = arg['initial'], strategyB = arg['opponent'],
                             verbose = arg['verbose'],pass_win_reward = arg['pass win reward'])
        arg['winrate'] = results['winnings'] / results['num_games']
        arg['model_logodds'] = results['final model']
        arg['model_probs'] = [round(logit(x)*100,1) for x in results['final model']]
        
        print(arg['name']," $",round(arg['winrate'],2),"  ",arg['model_probs'])    
 
    
    
    

In [368]:
firstargs = {'name':"Reward 1", 'decksize':6, 'alpha': 0.5,'initial':[0.01,0.01,0.01,0.01,0.01,0.01],'opponent':[],
            'pass win reward':1,'num_games':10**6,'num_updates':20,'verbose':False}
secondargs = {'name':"Reward 0", 'decksize':6, 'alpha':0.05,'initial':[0.01,0.01,0.01,0.01,0.01,0.01],'opponent':[],
             'pass win reward':0,'num_games':10**6,'num_updates':20,'verbose':False}
thirdargs = {'name':"Reward -1", 'decksize':6, 'alpha':0.005,'initial':[0.01,0.01,0.01,0.01,0.01,0.01],'opponent':[],
            'pass win reward':-1,'num_games':10**6,'num_updates':20,'verbose':False}
args = [firstargs,secondargs,thirdargs]
compare_outcomes(args)

Name  WinRate      Probvector
Reward 1  $ 0.23    [100.0, 97.1, 50.0, 0.4, 0.0, 0.0]
Reward 0  $ 0.22    [98.5, 99.9, 100.0, 100.0, 100.0, 100.0]
Reward -1  $ 0.22    [99.9, 99.7, 99.9, 99.0, 100.0, 100.0]


This seems too much.  We are now raising nearly all the time on any card.  But maybe this is indeed optimal vs. the base opponent.  Let's try out tournament.



In [371]:
"""Tournament.  Runs a tournament among several A and B strategies.

    Used some in testing, but not important in v1"""

num_games = 1000000
verbose = True

Astrategies = []
s = vectorstrat([.5,.5,.5,.5,.5,.5])
Astrategies += [{'strat':s,'name':"Random    "}]
s = vectorstrat([100.0, 97.1, 50.0, 0.4, 0.0, 0.0])
Astrategies += [{'strat':s,'name':"Original "}]
s = vectorstrat([99.9, 99.7, 99.9, 99.0, 100.0, 100.0])
Astrategies += [{'strat':s,'name':"P/W Reward -1"}]


Bstrategies = []
s = vectorstrat([.5,.5,.5,.5,.5,.5])
Bstrategies += [{'strat':s,'name':"Random    "}]
s = vectorstrat([0,0,0,1,1,1])
Bstrategies += [{'strat':s,'name':"Def Opponent"}]
s = vectorstrat([0,0,1,1,1,1])
Bstrategies += [{'strat':s,'name':"Simple 2"}]


for bstrat in Bstrategies:
    bstrat['profit'] = 0
    
for astrat in Astrategies:
    astrat['profit'] = 0
    for bstrat in Bstrategies:
        if verbose: print (astrat['name']," playing ",bstrat['name'])
        awins = challenge(num_games,astrat['strat'],bstrat['strat'],verbose = False)
        astrat['profit'] += awins
        bstrat['profit'] -= awins
        astrat[bstrat['name']] = awins
    astrat['average'] = astrat['profit'] / (num_games * len(Bstrategies))
    
for bstrat in Bstrategies:
    bstrat['average'] = bstrat['profit'] / (num_games * len(Astrategies))
    

print()
print ("Tournament Results -- winnings for Strategy A")
print("                        Strategy B")
print ("Strategy A    | ", end = "")
for b in Bstrategies:
    print(b['name'], end =" | ")
print("Total")
print("_______________________________________________")
for a in Astrategies:
    print(a['name'], end = "    |")
    for b in Bstrategies:
        print(a[b['name']], end = "         |")
    print(a['profit'])
print("Total         |", end = "")
for b in Bstrategies:
    print(-b['profit'], end = "         |")
print(-sum([b['profit'] for b in Bstrategies]))
    
        
    
                            


Random      playing  Random    
Random      playing  Def Opponent
Random      playing  Simple 2
Original   playing  Random    
Original   playing  Def Opponent
Original   playing  Simple 2
P/W Reward -1  playing  Random    
P/W Reward -1  playing  Def Opponent
P/W Reward -1  playing  Simple 2

Tournament Results -- winnings for Strategy A
                        Strategy B
Strategy A    | Random     | Def Opponent | Simple 2 | Total
_______________________________________________
Random        |251530         |-436         |-54866         |196228
Original     |280976         |-23670         |-206499         |50807
P/W Reward -1    |498159         |6         |-111341         |386824
Total         |1030665         |-24100         |-372706         |633859


Indeed, this is the best strategy vs the opponent.  I am now ready to go to having A and B learn together.

In [299]:
"""Compare learning.  

Not used in 1.1

   Compares two learning set-ups and graphs covergence stats.  
   Use to set hyperparameters (num_games, alpha) and test covergence
   Moving from ad-hoc review of performance to something better.

    TO DO (maybe)
    make graphics

"""
def av_vol(paramseries):
    # computes the moving GARCH vols of a timeseries of vectors, then takes the mean of the vector at each time.
    
    decayfactor = .5
    
    movingvars = [0 for i in paramseries[0]]
    previous = paramseries[0]
    av_vols =[]
    
    for newvalues in paramseries:
        movingvars = [decayfactor * oldmoving + (1 - decayfactor) * (old-new)**2 
                      for oldmoving,old,new in zip(movingvars,previous,newvalues)]
        movingvols = [x**0.5 for x in movingvars]
        av_vols += [(sum(movingvols)/len(movingvols))]
        previous = newvalues
        
    return av_vols
        
    


def compare (args,num_games = 10 ** 6, num_updates = 20):
    
    # args array of - dict.  'name' 'decksize' 'alpha' 'initial' 'opponent'
    
    tempwins = []
    games = []
    logodds = []
    performance = []
    stability = []
    
    for arg in args:
    
        arg['tempwins'] = []
        arg['games']= []
        arg['logodds'] = []
        arg['performance'] = []
        arg['stability'] = []
        
        results = train_A_logit(num_games, num_updates, 
                             alpha = arg['alpha'], decksize = arg['decksize'],
                             start_strategy = arg['initial'], strategyB = arg['opponent'],verbose = False)
  
        # transform results so they can be accessed easily -- Find better way to pass these.
    
        arg['tempwins'] += [[update['tempwins'] for update in results['training_updates']]]
        arg['games'] += [[update['games'] for update in results['training_updates']]]
        arg['logodds'] += [[update['logodds'] for update in results['training_updates']]]

        # compute performance and stability
        arg['performance'] += [ (win / (num_games/num_updates)) for win in arg['tempwins'][-1]]
        arg['stability'] += [av_vol(arg['logodds'][-1])]
 
    
    
    print ("                   ",end ="")
    for arg in args: 
        print(arg['name'],"               ",end = "")
    print("")
    print ("Games      ",end ="")
    for arg in args: 
        print("Performance   Stability   ",end ="")
    print("")
    
    #below is very ugly but cant figure out a better way
    if len(args) == 2:
        for g,p1,s1,p2,s2 in zip(args[0]['games'][0],args[0]['performance'],args[0]['stability'][0],
                                 args[1]['performance'],args[1]['stability'][0]):
            print(g,"     ",round(p1,4),"      ",round(s1,4),"     ",round(p2,4),"      ",round(s2,4))
    elif len(args)== 3:
        for g,p1,s1,p2,s2,p3,s3 in zip(args[0]['games'][0],args[0]['performance'],args[0]['stability'][0],
                                 args[1]['performance'],args[1]['stability'][0],
                                 args[2]['performance'],args[2]['stability'][0]):
            print(g,"     ",round(p1,4),"      ",round(s1,4),"     ",round(p2,4),"      ",round(s2,4),
                 "     ",round(p3,4),"      ",round(s3,4))
    else:
        for g,p1,s1,p2,s2,p3,s3,p4,s4 in zip(args[0]['games'][0],args[0]['performance'],args[0]['stability'][0],
                                 args[1]['performance'],args[1]['stability'][0],
                                 args[2]['performance'],args[2]['stability'][0],
                                 args[3]['performance'],args[3]['stability'][0],):
            print(g,"     ",round(p1,4),"      ",round(s1,4),"     ",round(p2,4),"      ",round(s2,4),
                 "     ",round(p3,4),"      ",round(s3,4), "     ",round(p4,4),"      ",round(s4,4))
